In [ ]:
# Install required packages
!pip install google-cloud-kms pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.9/267.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.5 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries

import os
import json
import uuid
import datetime
import base64
from google.cloud import kms
from google.colab import drive
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from pymongo import MongoClient
from pymongo.errors import ServerSelectionTimeoutError, ConnectionFailure

In [ ]:
# Mount Google Drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
# KeyManager Class Definition
class KeyManager:
    def __init__(self, mongo_uri, db_name="key_management", collection_name="keys", location_id='global', key_ring_id='colab-key-ring'):
        """
        Initialize the KeyManager with GCP KMS client and MongoDB connection.

        Args:
            mongo_uri (str): MongoDB connection string
            db_name (str): MongoDB database name
            collection_name (str): MongoDB collection name
            location_id (str): GCP location ID (e.g., 'global', 'us-east1')
            key_ring_id (str): GCP KMS key ring ID
        """
        self.location_id = location_id
        self.key_ring_id = key_ring_id

        # Initialize GCP KMS client
        self.kms_client = kms.KeyManagementServiceClient()

        # Create project path
        if 'GOOGLE_CLOUD_PROJECT' in os.environ:
            self.project_id = os.environ['GOOGLE_CLOUD_PROJECT']
        else:
            raise ValueError("GCP project ID not found. Please set the GOOGLE_CLOUD_PROJECT environment variable.")

        # Setup parent name for key ring
        self.parent = f"projects/{self.project_id}/locations/{self.location_id}"
        self.key_ring_name = f"{self.parent}/keyRings/{self.key_ring_id}"

        # Ensure key ring exists
        self._ensure_key_ring_exists()

        # Setup MongoDB connection
        try:
            self.client = MongoClient(mongo_uri, serverSelectionTimeoutMS=5000)
            # Test the connection
            self.client.server_info()
            self.db = self.client[db_name]
            self.keys_collection = self.db[collection_name]

            # Create indexes for better query performance
            self.keys_collection.create_index("key_id", unique=True)
            self.keys_collection.create_index("alias")
            print("Connected to MongoDB successfully")
        except (ServerSelectionTimeoutError, ConnectionFailure) as e:
            print(f"Failed to connect to MongoDB: {e}")
            raise

    def _ensure_key_ring_exists(self):
        """
        Ensure the key ring exists, creating it if necessary.
        """
        try:
            # Check if key ring exists
            request = kms.ListKeyRingsRequest(parent=self.parent)
            key_rings = [kr.name for kr in self.kms_client.list_key_rings(request=request)]

            if self.key_ring_name not in key_rings:
                # Create the key ring
                request = kms.CreateKeyRingRequest(
                    parent=self.parent,
                    key_ring_id=self.key_ring_id
                )
                self.kms_client.create_key_ring(request=request)
                print(f"Created key ring: {self.key_ring_id}")

        except Exception as e:
            print(f"Error ensuring key ring exists: {e}")
            raise

    def create_key(self, description, key_alias=None):
        """
        Create a new KMS key with optional alias.

        Args:
            description (str): Description for the key
            key_alias (str, optional): Alias name for the key

        Returns:
            dict: Key information including key_id
        """
        try:
            # Use key_alias as the crypto_key_id if provided, otherwise generate a UUID
            crypto_key_id = key_alias if key_alias else f"key-{uuid.uuid4().hex[:8]}"

            # Create key request
            request = kms.CreateCryptoKeyRequest(
                parent=self.key_ring_name,
                crypto_key_id=crypto_key_id,
                crypto_key=kms.CryptoKey(
                    purpose=kms.CryptoKey.CryptoKeyPurpose.ENCRYPT_DECRYPT,
                    version_template=kms.CryptoKeyVersionTemplate(
                        algorithm=kms.CryptoKeyVersion.CryptoKeyVersionAlgorithm.GOOGLE_SYMMETRIC_ENCRYPTION,
                    ),
                ),
            )

            # Create the key
            key = self.kms_client.create_crypto_key(request=request)

            # Extract the key name and ID
            key_name = key.name
            key_id = key_name.split('/')[-1]

            # Store metadata in MongoDB
            key_metadata = {
                'key_id': key_id,
                'key_name': key_name,
                'description': description,
                'alias': key_alias if key_alias else key_id,
                'created_at': datetime.datetime.now(),
                'rotated_at': None,
                'status': 'ACTIVE'
            }

            self.keys_collection.insert_one(key_metadata)

            return {
                'key_id': key_id,
                'key_name': key_name,
                'alias': key_metadata['alias'],
                'description': description,
                'created_at': key_metadata['created_at']
            }

        except Exception as e:
            print(f"Error creating key: {e}")
            return None

    def rotate_key(self, key_id_or_alias):
        """
        Rotate a key by creating a new primary version.

        Args:
            key_id_or_alias (str): KeyId or alias of the key to rotate

        Returns:
            dict: Updated key information
        """
        try:
            # Resolve key_id and key_name from alias if needed
            key_info = self._resolve_key_info(key_id_or_alias)
            if not key_info:
                return None

            key_id = key_info['key_id']
            key_name = key_info['key_name']

            # Create a new version of the key
            request = kms.CreateCryptoKeyVersionRequest(parent=key_name)
            new_version = self.kms_client.create_crypto_key_version(request=request)

            # Get version number from the name
            version_id = new_version.name.split('/')[-1]

            # Update metadata in MongoDB
            self.keys_collection.update_one(
                {'key_id': key_id},
                {'$set': {
                    'rotated_at': datetime.datetime.now(),
                    'current_version': version_id
                }}
            )

            return {
                'key_id': key_id,
                'key_name': key_name,
                'version_id': version_id,
                'rotated_at': datetime.datetime.now()
            }

        except Exception as e:
            print(f"Error rotating key: {e}")
            return None

    def list_keys(self):
        """
        List all keys managed by this program.

        Returns:
            list: List of key metadata
        """
        try:
            cursor = self.keys_collection.find({}, {'_id': 0})
            return list(cursor)
        except Exception as e:
            print(f"Error listing keys: {e}")
            return []

    def _resolve_key_info(self, key_id_or_alias):
        """
        Resolve key_id and key_name from alias if necessary.

        Args:
            key_id_or_alias (str): KeyId or alias name

        Returns:
            dict: Dictionary with key_id and key_name
        """
        # First check if it's a key ID stored in our MongoDB
        key = self.keys_collection.find_one({'key_id': key_id_or_alias})
        if key:
            return {'key_id': key['key_id'], 'key_name': key['key_name']}

        # Check if it's an alias in our MongoDB
        key = self.keys_collection.find_one({'alias': key_id_or_alias})
        if key:
            return {'key_id': key['key_id'], 'key_name': key['key_name']}

        # If not found in MongoDB, try to list keys from GCP
        try:
            request = kms.ListCryptoKeysRequest(parent=self.key_ring_name)
            for crypto_key in self.kms_client.list_crypto_keys(request=request):
                if crypto_key.name.split('/')[-1] == key_id_or_alias:
                    return {'key_id': key_id_or_alias, 'key_name': crypto_key.name}
        except Exception as e:
            print(f"Error resolving key info from GCP: {e}")

        return None

    def encrypt_data(self, key_id_or_alias, plaintext):
        """
        Encrypt data using the specified key.

        Args:
            key_id_or_alias (str): KeyId or alias of the key
            plaintext (str): Data to encrypt

        Returns:
            str: Base64-encoded ciphertext
        """
        try:
            key_info = self._resolve_key_info(key_id_or_alias)
            if not key_info:
                return None

            key_name = key_info['key_name']

            # Prepare the encryption request
            request = kms.EncryptRequest(
                name=key_name,
                plaintext=plaintext.encode('utf-8')
            )

            # Encrypt the data
            response = self.kms_client.encrypt(request=request)

            # Encode the ciphertext
            ciphertext = base64.b64encode(response.ciphertext).decode('utf-8')

            return ciphertext

        except Exception as e:
            print(f"Error encrypting data: {e}")
            return None

    def decrypt_data(self, ciphertext):
        """
        Decrypt data.

        Args:
            ciphertext (str): Base64-encoded ciphertext

        Returns:
            str: Decrypted plaintext
        """
        try:
            # Decode the ciphertext
            decoded_ciphertext = base64.b64decode(ciphertext)

            # Extract key name from ciphertext if available
            # Note: GCP KMS handles key identification automatically
            # You need to know which key was used for encryption

            # Get the most recently used key from MongoDB as a fallback
            recent_key = self.keys_collection.find_one(
                {'status': 'ACTIVE'},
                sort=[('created_at', -1)]
            )

            if not recent_key:
                print("No active keys found for decryption")
                return None

            key_name = recent_key['key_name']

            # Prepare the decryption request
            request = kms.DecryptRequest(
                name=key_name,
                ciphertext=decoded_ciphertext
            )

            # Decrypt the data
            response = self.kms_client.decrypt(request=request)

            return response.plaintext.decode('utf-8')

        except Exception as e:
            print(f"Error decrypting data: {e}")
            return None

    def disable_key(self, key_id_or_alias):
        """
        Disable a key (GCP KMS doesn't support deletion, only disabling).

        Args:
            key_id_or_alias (str): KeyId or alias of the key

        Returns:
            bool: Success status
        """
        try:
            key_info = self._resolve_key_info(key_id_or_alias)
            if not key_info:
                return False

            key_id = key_info['key_id']
            key_name = key_info['key_name']

            # Get the key
            key = self.kms_client.get_crypto_key(name=key_name)

            # Update the key to disable all new versions
            updated_key = kms.CryptoKey(
                name=key_name,
                primary_version=key.primary_version,
                purpose=key.purpose,
                version_template=kms.CryptoKeyVersionTemplate(
                    algorithm=key.version_template.algorithm,
                    protection_level=key.version_template.protection_level
                )
            )

            # Set the state for auto-rotation
            update_mask = {"paths": ["version_template.state"]}
            updated_key.version_template.state = kms.CryptoKeyVersion.CryptoKeyVersionState.DISABLED

            # Update the key
            request = kms.UpdateCryptoKeyRequest(
                crypto_key=updated_key,
                update_mask=update_mask
            )

            self.kms_client.update_crypto_key(request=request)

            # Update metadata in MongoDB
            self.keys_collection.update_one(
                {'key_id': key_id},
                {'$set': {
                    'status': 'DISABLED',
                    'disabled_at': datetime.datetime.now()
                }}
            )

            return True

        except Exception as e:
            print(f"Error disabling key: {e}")
            return False

In [ ]:
# Setup credentials function

def setup_credentials():
    # Create widgets for credentials
    gcp_service_account = widgets.FileUpload(
        description='GCP Service Account Key (JSON):',
        accept='.json',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='70%')
    )

    gcp_project = widgets.Text(
        description='GCP Project ID:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    gcp_location = widgets.Text(
        value='global',
        description='GCP Location:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    key_ring_id = widgets.Text(
        value='colab-key-ring',
        description='Key Ring ID:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    mongo_uri = widgets.Text(
        value='mongodb://localhost:27017/',
        description='MongoDB URI:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='70%')
    )

    mongo_db = widgets.Text(
        value='key_management',
        description='MongoDB Database:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    mongo_collection = widgets.Text(
        value='keys',
        description='MongoDB Collection:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%')
    )

    submit_button = widgets.Button(
        description='Set Credentials',
        button_style='primary',
        layout=widgets.Layout(width='20%')
    )

    output = widgets.Output()

    def on_submit_button_clicked(b):
        with output:
            clear_output()

            # Save service account file to local storage
            if len(gcp_service_account.value) > 0:
                file_name = list(gcp_service_account.value.keys())[0]
                content = gcp_service_account.value[file_name]['content']

                # Save to a temporary file
                service_account_path = '/tmp/gcp_service_account.json'
                with open(service_account_path, 'wb') as f:
                    f.write(content)

                # Set environment variable for authentication
                os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_path
                print(f"GCP service account key saved and set")
            else:
                print("No service account key provided. Authentication may fail.")

            # Set GCP project ID
            os.environ['GOOGLE_CLOUD_PROJECT'] = gcp_project.value
            print(f"GCP project ID set to: {gcp_project.value}")

            # Initialize key manager with MongoDB connection
            try:
                global key_manager
                key_manager = KeyManager(
                    mongo_uri=mongo_uri.value,
                    db_name=mongo_db.value,
                    collection_name=mongo_collection.value,
                    location_id=gcp_location.value,
                    key_ring_id=key_ring_id.value
                )
                print(f"MongoDB connection established: {mongo_db.value}.{mongo_collection.value}")
                print(f"Key ring: {key_ring_id.value} in location: {gcp_location.value}")
            except Exception as e:
                print(f"Failed to initialize key manager: {e}")

    submit_button.on_click(on_submit_button_clicked)

    # Display widgets
    display(widgets.VBox([
        widgets.HTML("<h3>GCP and MongoDB Credentials Setup</h3>"),
        gcp_service_account,
        gcp_project,
        gcp_location,
        key_ring_id,
        widgets.HTML("<h4>MongoDB Configuration</h4>"),
        mongo_uri,
        mongo_db,
        mongo_collection,
        submit_button,
        output
    ]))


In [ ]:
# Key management UI function

def create_key_management_ui():
    # Create tabs for different operations
    tab = widgets.Tab()
    tab_contents = []

    # Create Key Tab
    create_description = widgets.Text(
        description='Description:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='70%')
    )

    create_alias = widgets.Text(
        description='Alias (optional):',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='70%')
    )

    create_button = widgets.Button(
        description='Create Key',
        button_style='primary',
        layout=widgets.Layout(width='20%')
    )

    create_output = widgets.Output()

    def on_create_button_clicked(b):
        with create_output:
            clear_output()
            if not hasattr(globals(), 'key_manager'):
                print("Please set up credentials first!")
                return

            result = key_manager.create_key(create_description.value, create_alias.value)
            if result:
                print(f"Key created successfully:")
                print(f"Key ID: {result['key_id']}")
                print(f"Key Name: {result['key_name']}")
                print(f"Alias: {result['alias']}")
                print(f"Description: {result['description']}")
                print(f"Created at: {result['created_at']}")
            else:
                print("Failed to create key. Check logs for details.")

    create_button.on_click(on_create_button_clicked)

    create_container = widgets.VBox([
        widgets.HTML("<h3>Create New Key</h3>"),
        create_description,
        create_alias,
        create_button,
        create_output
    ])

    tab_contents.append(create_container)

    # List Keys Tab
    list_button = widgets.Button(
        description='List Keys',
        button_style='info',
        layout=widgets.Layout(width='20%')
    )

    list_output = widgets.Output()

    def on_list_button_clicked(b):
        with list_output:
            clear_output()
            if not hasattr(globals(), 'key_manager'):
                print("Please set up credentials first!")
                return

            keys = key_manager.list_keys()
            if keys:
                print(f"Found {len(keys)} keys:")
                for i, key in enumerate(keys, 1):
                    print(f"\nKey {i}:")
                    print(f"  Key ID: {key['key_id']}")
                    print(f"  Key Name: {key['key_name']}")
                    print(f"  Alias: {key['alias']}")
                    print(f"  Description: {key['description']}")
                    print(f"  Status: {key['status']}")
                    print(f"  Created: {key['created_at']}")
                    if key.get('rotated_at'):
                        print(f"  Rotated: {key['rotated_at']}")
            else:
                print("No keys found.")

    list_button.on_click(on_list_button_clicked)

    list_container = widgets.VBox([
        widgets.HTML("<h3>List All Keys</h3>"),
        list_button,
        list_output
    ])

    tab_contents.append(list_container)

    # Rotate Key Tab
    rotate_key = widgets.Text(
        description='Key ID or Alias:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='70%')
    )

    rotate_button = widgets.Button(
        description='Rotate Key',
        button_style='warning',
        layout=widgets.Layout(width='20%')
    )

    rotate_output = widgets.Output()

    def on_rotate_button_clicked(b):
        with rotate_output:
            clear_output()
            if not hasattr(globals(), 'key_manager'):
                print("Please set up credentials first!")
                return

            result = key_manager.rotate_key(rotate_key.value)
            if result:
                print(f"Key rotated successfully:")
                print(f"Key ID: {result['key_id']}")
                print(f"Key Name: {result['key_name']}")
                print(f"Version ID: {result['version_id']}")
                print(f"Rotated at: {result['rotated_at']}")
            else:
                print("Failed to rotate key. Check logs for details.")

    rotate_button.on_click(on_rotate_button_clicked)

    rotate_container = widgets.VBox([
        widgets.HTML("<h3>Rotate Key</h3>"),
        rotate_key,
        rotate_button,
        rotate_output
    ])

    tab_contents.append(rotate_container)

    # Encrypt Data Tab
    encrypt_key = widgets.Text(
        description='Key ID or Alias:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='70%')
    )

    encrypt_data = widgets.Textarea(
        description='Data to Encrypt:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='70%', height='100px')
    )

    encrypt_button = widgets.Button(
        description='Encrypt Data',
        button_style='success',
        layout=widgets.Layout(width='20%')
    )

    encrypt_output = widgets.Output()

    def on_encrypt_button_clicked(b):
        with encrypt_output:
            clear_output()
            if not hasattr(globals(), 'key_manager'):
                print("Please set up credentials first!")
                return

            ciphertext = key_manager.encrypt_data(encrypt_key.value, encrypt_data.value)
            if ciphertext:
                print("Data encrypted successfully:")
                print(f"\n{ciphertext}\n")
            else:
                print("Failed to encrypt data. Check logs for details.")

    encrypt_button.on_click(on_encrypt_button_clicked)

    encrypt_container = widgets.VBox([
        widgets.HTML("<h3>Encrypt Data</h3>"),
        encrypt_key,
        encrypt_data,
        encrypt_button,
        encrypt_output
    ])

    tab_contents.append(encrypt_container)

    # Decrypt Data Tab
    decrypt_data = widgets.Textarea(
        description='Data to Decrypt:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='70%', height='100px')
    )

    decrypt_button = widgets.Button(
        description='Decrypt Data',
        button_style='success',
        layout=widgets.Layout(width='20%')
    )

    decrypt_output = widgets.Output()

    def on_decrypt_button_clicked(b):
        with decrypt_output:
            clear_output()
            if not hasattr(globals(), 'key_manager'):
                print("Please set up credentials first!")
                return

            plaintext = key_manager.decrypt_data(decrypt_data.value)
            if plaintext:
                print("Data decrypted successfully:")
                print(f"\n{plaintext}\n")
            else:
                print("Failed to decrypt data. Check logs for details.")

    decrypt_button.on_click(on_decrypt_button_clicked)

    decrypt_container = widgets.VBox([
        widgets.HTML("<h3>Decrypt Data</h3>"),
        decrypt_data,
        decrypt_button,
        decrypt_output
    ])

    tab_contents.append(decrypt_container)

    # Disable Key Tab (GCP KMS doesn't support deletion, only disabling)
    disable_key = widgets.Text(
        description='Key ID or Alias:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='70%')
    )

    disable_button = widgets.Button(
        description='Disable Key',
        button_style='danger',
        layout=widgets.Layout(width='20%')
    )

    disable_output = widgets.Output()

    def on_disable_button_clicked(b):
        with disable_output:
            clear_output()
            if not hasattr(globals(), 'key_manager'):
                print("Please set up credentials first!")
                return

            success = key_manager.disable_key(disable_key.value)
            if success:
                print(f"Key disabled successfully: {disable_key.value}")
                print("Note: In GCP KMS, keys are disabled rather than deleted.")
            else:
                print("Failed to disable key. Check logs for details.")

    disable_button.on_click(on_disable_button_clicked)

    disable_container = widgets.VBox([
        widgets.HTML("<h3>Disable Key</h3>"),
        widgets.HTML("<p style='color:red'>WARNING: This will disable the key, making it unusable for new encryption operations.</p>"),
        disable_key,
        disable_button,
        disable_output
    ])

    tab_contents.append(disable_container)

    # Set tab titles and contents
    tab.children = tab_contents
    tab.titles = ('Create', 'List', 'Rotate', 'Encrypt', 'Decrypt', 'Disable')

    display(widgets.HTML("<h2>GCP KMS Key Management with MongoDB</h2>"))
    display(tab)

In [ ]:
# Display UI functions

def show_ui():
    # Main header
    display(HTML("<h1>Multicloud Key Management - GCP KMS with MongoDB Implementation</h1>"))
    display(HTML("<p>First, set up your GCP credentials and MongoDB connection details.</p>"))

    # Show credential setup
    setup_credentials()

    # Show the key management UI
    create_key_management_ui()


In [ ]:
# Run the UI

show_ui()

HTML(value='<h2>GCP KMS Key Management with MongoDB</h2>')